In [1]:
import scipy.integrate as spint
from numpy import cos,sin,exp, log as ln
import numpy as np
from numpy.linalg import inv,norm,cond,solve

# подинтегральная функция
def f(i,t):
    return exp(-i**2*t)/(1+i*t*sin(t)**2)

def T(N,f,a,b):
    if N==0:
        return (b-a)/2*(f(a)+f(b))
    else:
        g=2**N
        M=int(g/2)
        h=(b-a)/g
        def xk(k):
            return a+k*h
        return T(N-1,f,a,b)/2+h*sum([f(xk(2*k+1)) for k in range(1,M+1)])

def S(N,f,a,b):
    return (4*T(N,f,a,b)-T(N-1,f,a,b))/3

def B(N,f,a,b):
    return (16*S(N,f,a,b)-T(N-1,f,a,b))/15

def getIntegral(f,a,b,eps):
    N=2
    int1=B(1,f,a,b)
    int2=B(2,f,a,b)
    while abs(int2-int1)>eps:
        N+=1
        int1=int2
        int2=B(N,f,a,b)
    return [int2,N]


n=100

# Матрицы Ax=b
A=np.zeros([n,n],dtype=np.float32)
for j in range(0,n):
	i=j+1
	A[j][j]=(6*i**2+3)
for j in range(1,n-1):
	i=j+1
	A[j][j-1]=(2*i+1)
	A[j][j+1]=(4*i+2)
A[0][0]=A[n-1][n-1]=1
A[0][1]=-0.1
A[n-1][n-2]=-0.5

b=np.zeros(n,dtype=np.float32)
for j in range(1,n-1):
	epsi=0.01
	def rightIntegral(t):
		return f(j+1,t)
	b[j]=getIntegral(rightIntegral,0,1,epsi)[0]

L=np.tril(A, k=-1) #triangular_lower_no_diag matrix from A
R=np.triu(A, k=1) # triangular_upper_no_diag matrix from A
D=np.diag(np.diag(A)) #diag matrix from A


x9=solve(A,b)

d,c,a=np.diag(A,k=0),np.diag(A,k=1),np.diag(A,k=-1)
M,L,x=np.zeros(n),np.zeros(n),np.zeros(n)

# Прямая прогонка 
for i in range(0,n):
    if i==0:
        M[i]=-c[0]/d[0]
        L[i]=b[0]/d[0]
    else:
        M[i]=(b[i-1]-M[i-1]*a[i-2])/(a[i-2]*L[i-1]+d[i-1])
        L[i]=-c[i-1]/(a[i-2]*L[i-1]+d[i-1])

# Обратная прогонка 
x[n-1]=(b[n-1]-M[n-1]*a[n-2])/(a[n-2]*L[n-1]+d[n-1])

for i in reversed(range(0,n-1)):
    x[i]=x[i+1]*L[i+1]+M[i+1]

print(x)
print(x9)

[ 4.93573393e-02 -6.42661484e-03  1.50319090e-03  1.06328855e-04
  8.15357075e-05  4.08826518e-05  2.78004672e-05  2.09329977e-05
  1.66366336e-05  1.36078048e-05  1.13487696e-05  9.61015571e-06
  8.24227077e-06  7.14667008e-06  6.25566018e-06  5.52134145e-06
  4.90904377e-06  4.39317405e-06  3.95450915e-06  3.57838519e-06
  3.25345966e-06  2.97084840e-06  2.72351313e-06  2.50581986e-06
  2.31321568e-06  2.14198919e-06  1.98909055e-06  1.85199488e-06
  1.72859751e-06  1.61713275e-06  1.51611047e-06  1.42426603e-06
  1.34052060e-06  1.26394927e-06  1.19375551e-06  1.12925030e-06
  1.06983523e-06  1.01498855e-06  9.64253674e-07  9.17229655e-07
  8.73563256e-07  8.32942287e-07  7.95090013e-07  7.59760432e-07
  7.26734266e-07  6.95815555e-07  6.66828741e-07  6.39616172e-07
  6.14035955e-07  5.89960102e-07  5.67272928e-07  5.45869657e-07
  5.25655210e-07  5.06543152e-07  4.88454759e-07  4.71318213e-07
  4.55067884e-07  4.39643703e-07  4.24990605e-07  4.11058039e-07
  3.97799528e-07  3.85172